## Part 3: Pulling and Downloading Random Item Links

In [2]:
%%capture
!pip install -U pystac-client

In [3]:
from maap.maap import MAAP
from pprint import pprint
maap = MAAP()

from pystac_client import Client

In [4]:
# STAC API root URL
URL = 'https://wssn144yw1.execute-api.us-west-2.amazonaws.com/'
cat = Client.open(URL)
cat

ID: stac-fastapi
Title: MAAP STAC API (dev)
Description: STAC API for the MAAP STAC system.
type: Catalog
"conformsTo: ['http://www.opengis.net/spec/ogcapi-features-1/1.0/conf/core', 'https://api.stacspec.org/v1.0.0-rc.1/item-search#filter:basic-cql', 'https://api.stacspec.org/v1.0.0-rc.1/item-search#filter', 'https://api.stacspec.org/v1.0.0-rc.1/item-search#filter:cql-text', 'https://api.stacspec.org/v1.0.0-rc.1/item-search#context', 'https://api.stacspec.org/v1.0.0-rc.1/ogcapi-features', 'https://api.stacspec.org/v1.0.0-rc.1/core', 'https://api.stacspec.org/v1.0.0-rc.1/item-search#fields', 'https://api.stacspec.org/v1.0.0-rc.1/item-search', 'http://www.opengis.net/spec/ogcapi-features-3/1.0/conf/filter', 'http://www.opengis.net/spec/ogcapi-features-1/1.0/conf/oas30', 'http://www.opengis.net/spec/ogcapi-features-3/1.0/conf/features-filter', 'https://api.stacspec.org/v1.0.0-rc.1/item-search#sort', 'https://api.stacspec.org/v1.0.0-rc.1/collections', 'https://api.stacspec.org/v1.0.0-rc.1/item-search#query', 'http://www.opengis.net/spec/ogcapi-features-1/1.0/conf/geojson']"
https://raw.githubusercontent.com/radiantearth/stac-api-spec/v1.0.0-rc.1/fragments/context/json-schema/schema.json
ID: ESACCI_Biomass_L4_AGB_V3_100m_2018
"Title: ESA CCI above-ground biomass product level 4, year 2018"
"Description: This dataset comprises estimates of forest above-ground biomass for the years 2010, 2017 and 2018. They are derived from a combination of Earth observation data, depending on the year, from the Copernicus Sentinel-1 mission, Envisat’s ASAR instrument and JAXA’s Advanced Land Observing Satellite (ALOS-1 and ALOS-2), along with additional information from Earth observation sources. The data has been produced as part of the European Space Agency's (ESA's) Climate Change Initiative (CCI) programme by the Biomass CCI team."
type: Collection
"title: ESA CCI above-ground biomass product level 4, year 2018"


In [5]:
# automating a list to pull the collection ids

collection_names = cat.get_all_collections()

collection_list = []
        
def get_collections(collection_names):
    for collection_names in collection_names:
        collection_list.append(collection_names.id)
    print(collection_list)
    
#def get_collections(collection_names):
    #return [collection_names.id for collection_names in collection_names]

get_collections(collection_names)
#print(len(collection_list))

['ESACCI_Biomass_L4_AGB_V3_100m_2018', 'ESACCI_Biomass_L4_AGB_V3_100m_2017', 'ESACCI_Biomass_L4_AGB_V3_100m_2010', 'Landsat8_SurfaceReflectance', 'Global_PALSAR2_PALSAR_FNF', 'Global_Forest_Change_2000-2017', 'AFRISAR_DLR2', 'AfriSAR_UAVSAR_KZ', 'AfriSAR_UAVSAR_Ungeocoded_Covariance', 'AfriSAR_UAVSAR_Normalization_Area', 'AfriSAR_UAVSAR_Geocoded_SLC', 'AfriSAR_UAVSAR_Geocoded_Covariance', 'GlobCover_09', 'GlobCover_05_06', 'GEDI_CalVal_Field_Data', 'AfriSAR_UAVSAR_Coreg_SLC', 'GEDI_CalVal_Lidar_Data_Compressed', 'GEDI_CalVal_Lidar_Data', 'ALOS_PSR_RTC_HIGH', 'ABoVE_UAVSAR_PALSAR', 'AFRISAR_DLR', 'BIOSAR1']


In [31]:
# choosing a random collection from stac to test

import random

col_id = random.choice(collection_list)
print(col_id)

collection = cat.get_collection(col_id)

collection

ESACCI_Biomass_L4_AGB_V3_100m_2010


ID: ESACCI_Biomass_L4_AGB_V3_100m_2010
"Title: ESA CCI above-ground biomass product level 4, year 2010"
"Description: This dataset comprises estimates of forest above-ground biomass for the years 2010, 2017 and 2018. They are derived from a combination of Earth observation data, depending on the year, from the Copernicus Sentinel-1 mission, Envisat’s ASAR instrument and JAXA’s Advanced Land Observing Satellite (ALOS-1 and ALOS-2), along with additional information from Earth observation sources. The data has been produced as part of the European Space Agency's (ESA's) Climate Change Initiative (CCI) programme by the Biomass CCI team."
type: Collection
"title: ESA CCI above-ground biomass product level 4, year 2010"
item_assets: {}
ID: S50W080_ESACCI-BIOMASS-L4-AGB_SD-MERGED-100m-2010-fv3.0
"Bounding Box: [-60.0, -80.0, -50.0, -70.0]"
Datetime: 2010-01-01 00:00:00+00:00
updated: 2021-09-02T08:17:05+00:00
datetime: 2010-01-01T00:00:00+00:00


In [32]:
# pulling five items in the collection and listing their links

items = collection.get_items()
asset_link = lambda item: item.assets['data'].href
links = [asset_link(item) for item in list(items)[:5]]
print(links)

['https://bmap-catalogue-data.oss.eu-west-0.prod-cloud-ocb.orange-business.com/esacci/ESACCI_Biomass_L4_AGB_V3_100m/2010/S50W080_ESACCI-BIOMASS-L4-AGB_SD-MERGED-100m-2010-fv3.0.tif', 'https://bmap-catalogue-data.oss.eu-west-0.prod-cloud-ocb.orange-business.com/esacci/ESACCI_Biomass_L4_AGB_V3_100m/2010/S50W080_ESACCI-BIOMASS-L4-AGB-MERGED-100m-2010-fv3.0.tif', 'https://bmap-catalogue-data.oss.eu-west-0.prod-cloud-ocb.orange-business.com/esacci/ESACCI_Biomass_L4_AGB_V3_100m/2010/S50W070_ESACCI-BIOMASS-L4-AGB_SD-MERGED-100m-2010-fv3.0.tif', 'https://bmap-catalogue-data.oss.eu-west-0.prod-cloud-ocb.orange-business.com/esacci/ESACCI_Biomass_L4_AGB_V3_100m/2010/S50W070_ESACCI-BIOMASS-L4-AGB-MERGED-100m-2010-fv3.0.tif', 'https://bmap-catalogue-data.oss.eu-west-0.prod-cloud-ocb.orange-business.com/esacci/ESACCI_Biomass_L4_AGB_V3_100m/2010/S50W060_ESACCI-BIOMASS-L4-AGB_SD-MERGED-100m-2010-fv3.0.tif']


In [33]:
# removing s3 bucket from URL if applicable

working_list = []

for link in links:
    if link.startswith("s3"):
        working_list.append("/".join(link.split("/")[3:]))
    elif link.startswith("https"):
        working_list.append(link)

working_list

['https://bmap-catalogue-data.oss.eu-west-0.prod-cloud-ocb.orange-business.com/esacci/ESACCI_Biomass_L4_AGB_V3_100m/2010/S50W080_ESACCI-BIOMASS-L4-AGB_SD-MERGED-100m-2010-fv3.0.tif',
 'https://bmap-catalogue-data.oss.eu-west-0.prod-cloud-ocb.orange-business.com/esacci/ESACCI_Biomass_L4_AGB_V3_100m/2010/S50W080_ESACCI-BIOMASS-L4-AGB-MERGED-100m-2010-fv3.0.tif',
 'https://bmap-catalogue-data.oss.eu-west-0.prod-cloud-ocb.orange-business.com/esacci/ESACCI_Biomass_L4_AGB_V3_100m/2010/S50W070_ESACCI-BIOMASS-L4-AGB_SD-MERGED-100m-2010-fv3.0.tif',
 'https://bmap-catalogue-data.oss.eu-west-0.prod-cloud-ocb.orange-business.com/esacci/ESACCI_Biomass_L4_AGB_V3_100m/2010/S50W070_ESACCI-BIOMASS-L4-AGB-MERGED-100m-2010-fv3.0.tif',
 'https://bmap-catalogue-data.oss.eu-west-0.prod-cloud-ocb.orange-business.com/esacci/ESACCI_Biomass_L4_AGB_V3_100m/2010/S50W060_ESACCI-BIOMASS-L4-AGB_SD-MERGED-100m-2010-fv3.0.tif']

In [34]:
# importing and getting AWS access

import boto3
import requests

from maap.maap import MAAP


maap = MAAP(maap_host='api.ops.maap-project.org')
credentials = maap.aws.requester_pays_credentials()

boto3_session = boto3.Session(
    aws_access_key_id=credentials['aws_access_key_id'],
    aws_secret_access_key=credentials['aws_secret_access_key'],
    aws_session_token=credentials['aws_session_token']
)

# checking to see if given collection objects exist

client = boto3.client('s3')
session = requests.Session() 

bucket_name = 'nasa-maap-data-store'

for key in working_list:
    if key.startswith("https"):
        response = session.head(key)
        print(response.headers)
    else:
        response = client.head_object(
            Bucket=bucket_name,
            Key=key,
        )

    print(response)

{'Server': 'OBS', 'x-reserved': 'amazon, aws and amazon web services are trademarks or registered trademarks of Amazon Technologies, Inc', 'x-amz-request-id': '00000188210FFF08941191854B38B9F8', 'Accept-Ranges': 'bytes', 'ETag': '"6e705914327a21f20e5416fe4ec71b3d"', 'Last-Modified': 'Thu, 19 Aug 2021 08:28:25 GMT', 'x-amz-meta-s3cmd-attrs': 'atime:1629296321/ctime:1629357484/gid:1002/gname:cloud/md5:6e705914327a21f20e5416fe4ec71b3d/mode:33204/mtime:1629296322/uid:1002/uname:cloud', 'Content-Type': 'image/tiff', 'x-amz-id-2': '32AAAQAAEAABAAAQAAEAABAAAQAAEAABCSEfKyCHjPiVmsDVWIS3Ax9zQuHLfng2', 'Date': 'Mon, 15 May 2023 20:17:56 GMT', 'Content-Length': '13293902'}
<Response [200]>
{'Server': 'OBS', 'x-reserved': 'amazon, aws and amazon web services are trademarks or registered trademarks of Amazon Technologies, Inc', 'x-amz-request-id': '00000188210FFFB8941191A508D13603', 'Accept-Ranges': 'bytes', 'ETag': '"01a0099bcc94674c8ad93ec65626d3ba"', 'Last-Modified': 'Thu, 19 Aug 2021 08:28:24 GM